In [10]:
%pip install transformers
%pip install thefuzz
%pip install fuzzywuzzy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


- https://en.wikipedia.org/wiki/Judo

- https://en.wikipedia.org/wiki/List_of_judo_techniques

- https://en.wikipedia.org/wiki/List_of_judoka

- https://martialarts.fandom.com/wiki/Judo

- https://chas-ma.com/JudoManual/Chapter_2%28HistoryofJudo%29.pdf

- https://www.ijf.org/history

- https://blackbelttrek.com/judo-vs-jiu-jitsu-the-ultimate-comparison/

In [11]:
import json
import os
from transformers import pipeline
from thefuzz import fuzz

# Lade das vortrainierte Modell
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

data_dir = "data-sources"
context_file = "complete_context.txt"

# Lese alle .txt-Dateien im Verzeichnis data-sources ein
context_parts = []
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            context_parts.append(f.read())

# Verbinde alle Texte zu einem Gesamttext
context = "\n".join(context_parts)

# Speichere den gesamten Kontext in eine Datei
with open(context_file, "w", encoding="utf-8") as f:
    f.write(context)

Device set to use cpu


In [12]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Ensure context is loaded
if 'context' not in locals():
	with open(context_file, "r", encoding="utf-8") as f:
		context = f.read()
print('Context length (chars): ', len(context))

tokens = tokenizer.tokenize(context)


# Calculate total token length
avg_length = sum(len(token.strip('Ġ')) for token in tokens)/len(tokens)
print(f"Avg Token length: {avg_length:.2f}")
print(f"Total Tokens: {len(tokens)}")

Context length (chars):  141653
Avg Token length: 2.93
Total Tokens: 41611


In [17]:
import json
import random

# Load questions
with open("question-sets/q_v2.json", "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# Select 20 random questions
random.seed(42)  # For reproducibility
sampled_questions = random.sample(questions_data, 20)

THRESHOLD = 60
correct = 0
total = len(sampled_questions)  # Only 20 questions

# Test the randomly selected questions
for item in sampled_questions:
    question = item["question"]
    expected_answer = item["answer"]

    # Query the model
    QA_input = {
        'question': question,
        'context': context
    }
    result = nlp(QA_input)
    predicted_answer = result["answer"]

    # Calculate answer similarity (using thefuzz)
    similarity = fuzz.ratio(predicted_answer.lower(), expected_answer.lower())
    
    # Compare with threshold
    if similarity >= THRESHOLD:
        correct += 1
        print(f"✅ Question: {question}")
        print(f"   Expected: {expected_answer}")
        print(f"   Received: {predicted_answer}")
        print(f"   Similarity: {similarity}%\n")
    else:
        print(f"❌ Question: {question}")
        print(f"   Expected: {expected_answer}")
        print(f"   Received: {predicted_answer}")
        print(f"   Similarity: {similarity}%\n")

# Calculate accuracy
accuracy = (correct / total) * 100
print(f"\n✅ Accuracy: {accuracy:.2f}% ({correct}/{total} correct)")

C:\Users\anton\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\pipelines\question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


❌ Question: What was judo’s inclusion status for the 1940 Tokyo Olympics?
   Expected: demonstration sport
   Received: philosophy
   Similarity: 14%

✅ Question: What is a minor point called?
   Expected: yuko
   Received: yuko
   Similarity: 100%

❌ Question: What is the name of the school Kanō Jigorō established?
   Expected: Kōdōkan
   Received: Kōdōkan Judo Institute
   Similarity: 48%

✅ Question: What does Jita‑Kyōei mean?
   Expected: mutual welfare and benefit
   Received: mutual welfare and benefit
   Similarity: 100%

✅ Question: In what year did women's judo debut as a demonstration sport at the Olympics?
   Expected: 1988
   Received: 1988
   Similarity: 100%

✅ Question: When did men's judo first appear at the Olympics?
   Expected: 1964
   Received: 1964
   Similarity: 100%

✅ Question: What is the Japanese term for grappling techniques?
   Expected: katame-waza
   Received: Katame-waza
   Similarity: 100%

✅ Question: What does waza-ari mean?
   Expected: half point
   